In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from ext.lab2im import utils
from SynthSeg.brain_generator import read_tfrecords
import tensorflow as tf

## Peek into data

Data downloaded from https://datashare.mpcdf.mpg.de/f/348054516

In [ ]:
image_nii = nib.load("t1w_pdw_validation_pair/t1_pdw_scaled_256.nii")

In [ ]:
image = img_nii.get_fdata()
image.shape

In [ ]:
plt.imshow(image[100,..., 1])

In [ ]:
labels_nii = nib.load("t1w_pdw_validation_pair/label_256.nii")

In [ ]:
plt.imshow(image[100,..., 0], alpha=1.0)
plt.imshow(labels[100, ...], alpha=0.5)

## Map labels

In [ ]:
class LabelMapping:
    def __init__(self, cfg_path):
        cfg_path = Path(cfg_path)
        with cfg_path.open() as file:
            cfg = yaml.safe_load(file)
        output_labels = cfg["output_labels"]
        self._mapping = utils.get_mapping_lut(np.unique(output_labels))

    def lab2idx(self, label: int) -> int:
        try:
            return self._mapping[label]
        except IndexError:
            return 0

    def idx2lab(self, idx: int):
        return np.nonzero(self._mapping == idx)[0][0]

In [ ]:
label_mapping = LabelMapping("./generator.yml")

In [ ]:
labels = labels_nii.get_fdata().astype(np.int32)
np.unique(labels), len(np.unique(labels))

In [ ]:
labels_mapped = np.array(list(map(label_mapping.lab2idx, labels.flatten()))).reshape(labels.shape)

In [ ]:
np.unique(labels_mapped)

## Create TFRecords

In [ ]:
compression_type = None
file = "./t1w_pdw_validation_pair.tfrecord"

with tf.io.TFRecordWriter(
    str(file), options=tf.io.TFRecordOptions(compression_type=compression_type)
) as writer:
    # create tf example
    features = {
        "image": tf.train.Feature(
            bytes_list=tf.train.BytesList(
                value=[tf.io.serialize_tensor(image.astype(np.float32)).numpy()]
            )
        ),
        "labels": tf.train.Feature(
            bytes_list=tf.train.BytesList(
                value=[tf.io.serialize_tensor(labels_mapped).numpy()]
            )
        ),
    }  

    example = tf.train.Example(features=tf.train.Features(feature=features))

    # write to file
    writer.write(example.SerializeToString())

In [ ]:
# sanity check
ds = read_tfrecords([file])
img, lab = next(iter(ds))
np.allclose(img.numpy(), image), np.allclose(lab.numpy(), labels_mapped)

Output file uploaded to https://datashare.mpcdf.mpg.de/f/348054516

## ... Playground

In [ ]:
img_nii = nib.load("../01_from_datashare/T1w.nii")

In [ ]:
img = img_nii.get_fdata()

In [ ]:
img.max()

In [ ]:
plt.hist(crop(test_vol).flatten(), bins=100);

In [ ]:
test_vol = nib.load("T1w_rescaled_for_segmentation.nii").get_fdata()

In [ ]:
def crop(volume):
    max_idxs = []
    for dim in volume.shape:
        max_idxs.append(dim - 256)
        
    slices = []
    for max_idx in max_idxs:
        rand_int = np.random.randint(0, max_idx)
        slices.append(slice(rand_int, rand_int + 256))

    return volume[slices[0], slices[1], slices[2]]

In [ ]:
iterator = iter(train_ds)

In [ ]:
train_img, label_img = next(iterator)

In [ ]:
train_img.shape, train_img[..., 0].numpy().sum()

In [ ]:
plt.hist(train_img.numpy()[..., 0].flatten(), bins=100);
plt.hist(img[...,0].flatten(), bins=100, alpha=0.3);

In [ ]:
from SynthSeg.brain_generator import read_tfrecords

In [ ]:
train_ds = read_tfrecords(["./train_tfrecords/000000.tfrecord"])

In [ ]:
plt.imshow(seg[100,...], alpha=0.5)
plt.imshow(img_rs[100, ...], alpha=0.5)

In [ ]:
img_rs_nii = nib.load("../t1w_pdw_config/T1w_rescaled_for_segmentation.nii")
img_rs = img_rs_nii.get_fdata()
img_rs.max()

Run segmentation as described in `approach.ipynb`

In [ ]:
seg_nii = nib.load("T1w_segmentations2.nii")
seg = seg_nii.get_fdata()

In [ ]:
seg_nii.header.get_data_shape()

In [ ]:
labels = np.unique(seg)
labels, len(labels)

In [ ]:
img_rs.shape, seg.shape

In [ ]:
from SynthSeg.analysis.contrast_analysis import clip_and_rescale_nifti

root_dir = "/home/david/mpcdf/cbs/segmentation/SynthSeg"
input_file = f"{root_dir}/data/cbs/01_from_datashare/T1w.nii"
output_file = f"{root_dir}/data/cbs/t1w_pdw_config/T1w_rescaled_for_segmentation.nii"

clip_and_rescale_nifti(
    nifti_file=input_file,
    out_file=output_file,
    min_clip=0.0,
    max_clip=2000,
    min_out=0.0,
    max_out=1.0
)